In [11]:
import requests
import json
import sqlite3
from datetime import datetime

# Function to fetch CPI data from BLS API
def fetch_cpi_data(series_ids, start_year, end_year, api_key=None):
    url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
    headers = {'Content-type': 'application/json'}
    data = json.dumps({
        "seriesid": series_ids,
        "startyear": start_year,
        "endyear": end_year
    })
    
    try:
        response = requests.post(url, data=data, headers=headers)
        response.raise_for_status()
        json_data = response.json()
        if 'Results' in json_data and 'series' in json_data['Results']:
            return json_data['Results']['series']
        else:
            print("No results found in the response.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON: {e}")
        return None

# Function to process the data and return it in the desired format
def process_data(series_data):
    time_series = {}
    for series in series_data:
        for item in series['data']:
            year = int(item['year'])
            period = item['period']
            value = item['value']
            
            if period.startswith('M'):
                month = int(period[1:])
                date_str = f"{year}-{month:02d}"  # Assuming the data is for the 1st day of each month
                time_series[date_str] = {"value": value}
    return time_series

# Function to create and insert CPI data into the database
def insert_cpi_data(time_series):
    conn = sqlite3.connect('finance1.db')
    cursor = conn.cursor()
    
    # Create the economics_cpi table if it does not exist
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS economics_cpi (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT UNIQUE NOT NULL,
        value REAL NOT NULL
    )
    ''')
    
    # Insert the CPI data into the table
    for date, data in time_series.items():
        try:
            cursor.execute('''
            INSERT INTO economics_cpi (date, value)
            VALUES (?, ?)
            ''', (date, data['value']))
        except sqlite3.IntegrityError:
            # Duplicate entry detected, skipping insertion
            pass
    
    conn.commit()
    conn.close()

# Main function
def main():
    api_key = 'my_key'  # Optional: Add your API key here
    series_ids = ['CUUR0000SA0']  # Add more series IDs as needed
    start_year = '2024'
    end_year = '2024'
    
    # Fetch and process CPI data
    series_data = fetch_cpi_data(series_ids, start_year, end_year, api_key)
    if series_data:
        time_series = process_data(series_data)
        insert_cpi_data(time_series)

if __name__ == "__main__":
    main()

In [1]:

import pandas as pd
import sqlite3

# Step 2: Connect to SQLite database named 'library.db'
conn = sqlite3.connect('finance1.db')

# Step 3: Create a cursor object
cursor = conn.cursor()

cursor.execute('''
    SELECT * FROM economics_cpi
''')


# Step 5: Fetch all rows from the executed query
rows = cursor.fetchall()

# Step 6: Get the field names
field_names = [description[0] for description in cursor.description]

# Step 7: Print the field names
df = pd.DataFrame(rows, columns=field_names)

# Step 9: Close the connection
conn.close()


In [2]:
import pandas as pd
import sqlite3

# Step 2: Connect to SQLite database named 'library.db'
conn = sqlite3.connect('finance1.db')

# Step 3: Create a cursor object
cursor = conn.cursor()

cursor.execute('''
    SELECT * FROM Stocks
''')

# Step 5: Fetch all rows from the executed query
rows = cursor.fetchall()

# Step 6: Get the field names
field_names = [description[0] for description in cursor.description]

# Step 7: Print the field names
stocknames_df = pd.DataFrame(rows, columns=field_names)

#stocknames_df = stocknames_df[['stock_id','symbol']]

cursor.execute('''
    SELECT * FROM StockPrices
''')

# Step 5: Fetch all rows from the executed query
rows = cursor.fetchall()

# Step 6: Get the field names
field_names = [description[0] for description in cursor.description]

# Step 7: Print the field names
stockprices_df = pd.DataFrame(rows, columns=field_names)

# Step 9: Close the connection
conn.close()


In [3]:
# Merge the two DataFrames on 'stock_id'
merged_df = pd.merge(stockprices_df, stocknames_df, on='stock_id')

In [4]:
merged_df = merged_df[['date', 'close', 'volume', 'symbol']]

In [5]:
import pandas as pd
import sqlite3

# Step 2: Connect to SQLite database named 'library.db'
conn = sqlite3.connect('finance1.db')

# Step 3: Create a cursor object
cursor = conn.cursor()

cursor.execute('''
    SELECT * FROM economics_cpi
''')

# Step 5: Fetch all rows from the executed query
rows = cursor.fetchall()

# Step 6: Get the field names
field_names = [description[0] for description in cursor.description]

# Step 7: Print the field names
economics_cpi_df = pd.DataFrame(rows, columns=field_names)

# Step 9: Close the connection
conn.close()


In [6]:
economics_cpi_df['date'] = pd.to_datetime(economics_cpi_df['date'])
# Extract year and month from the date columns
economics_cpi_df['year'] = economics_cpi_df['date'].dt.year
economics_cpi_df['month'] = economics_cpi_df['date'].dt.month

In [7]:
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month

In [8]:
merged_data_with_cpi = pd.merge(merged_df, economics_cpi_df, on=['year', 'month'], how='left')


In [9]:
merged_data_with_cpi = merged_data_with_cpi.dropna()

In [10]:
merged_data_with_cpi = merged_data_with_cpi[['date_x', 'close', 'volume', 'symbol', 'value']]

In [11]:
#df.rename(columns={'close': 'stock_price'}, inplace=True)
merged_data_with_cpi.rename(columns={'close': 'stock_price'}, inplace=True)
merged_data_with_cpi.rename(columns={'value': 'cpi_value'}, inplace=True)

In [ ]:
tsla_rows = df[df['symbol'] == 'TSLA']

merged_data_with_cpi 
tsla_r

In [12]:
tsla_rows = merged_data_with_cpi[merged_data_with_cpi['symbol'] == 'TSLA']

In [14]:
tsla_rows.to_csv('tsla_rows.csv', index=False)

In [15]:
tsla_rows

,date_x,stock_price,volume,symbol,cpi_value
111,2024-06-28,197.88,95438068,TSLA,314.175
112,2024-06-27,197.42,72746521,TSLA,314.175
113,2024-06-26,196.37,95737066,TSLA,314.175
114,2024-06-25,187.35,63678265,TSLA,314.175
115,2024-06-24,182.58,61992070,TSLA,314.175
...,...,...,...,...,...
195,2024-02-28,202.04,99806173,TSLA,310.326
196,2024-02-27,199.73,108645412,TSLA,310.326
197,2024-02-26,199.40,111747116,TSLA,310.326
198,2024-02-23,191.97,78841917,TSLA,310.326


In [18]:

def calculate_correlation(merged_data_with_cpi, symbol):
    """
    Calculate the correlation between 'close' and 'value' columns for a given symbol.
    
    Parameters:
    merged_data_with_cpi (pd.DataFrame): The DataFrame containing the data.
    symbol (str): The symbol to filter the DataFrame.
    
    Returns:
    pd.DataFrame: The correlation matrix between 'close' and 'value' columns.
    """
    # Filter the DataFrame for the given symbol
    symbol_df = merged_data_with_cpi[merged_data_with_cpi['symbol'] == symbol]
    
    # Calculate the correlation between 'close' and 'value' columns
    correlation = symbol_df[['stock_price', 'cpi_value']].corr()
    
    return correlation

# Example usage
# merged_data_with_cpi is your DataFrame
# symbol is the stock symbol you are interested in, e.g., 'TSLA'
correlation_data = calculate_correlation(merged_data_with_cpi, 'TSLA')
correlation_data

,stock_price,cpi_value
stock_price,1.000000,-0.304725
cpi_value,-0.304725,1.000000


In [38]:
symbols = ['AAPL', 'MSFT', 'TSLA', 'NVDA']  # Example stock symbols

In [39]:
import pandas as pd
correlation_df = pd.DataFrame(columns=['Symbol', 'Correlation'])

,Symbol,Correlation


In [48]:
import pandas as pd
correlation_df = pd.DataFrame(columns=['Symbol', 'Correlation'])
for symbol in symbols: 
    correlation_data = calculate_correlation(merged_data_with_cpi, symbol)
    new_row = {'Symbol': symbol,  'Correlation': correlation_data.iloc[0, 1]}
    new_row_df = pd.DataFrame([new_row])
    correlation_df = pd.concat([correlation_df, new_row_df], ignore_index=True)



In [49]:
correlation_df

,Symbol,Correlation
0,AAPL,0.422638
1,MSFT,0.356507
2,TSLA,-0.304725
3,NVDA,-0.176158


In [50]:
correlation_df.set_index('Symbol', inplace=True)


In [51]:
correlation_df

,Correlation
Symbol,
AAPL,0.422638
MSFT,0.356507
TSLA,-0.304725
NVDA,-0.176158


In [23]:
correlation_data = calculate_correlation(merged_data_with_cpi, 'AAPL')
correlation_data

,stock_price,cpi_value
stock_price,1.000000,0.422638
cpi_value,0.422638,1.000000


In [24]:
second_item_first_row = correlation_data.iloc[0, 1]


In [25]:
second_item_first_row

0.4226379392218521

In [22]:
correlation_data = calculate_correlation(merged_data_with_cpi, 'MSFT')
correlation_data

,stock_price,cpi_value
stock_price,1.000000,0.356507
cpi_value,0.356507,1.000000


In [23]:
correlation_data = calculate_correlation(merged_data_with_cpi, 'TSLA')
correlation_data

,stock_price,cpi_value
stock_price,1.000000,-0.304725
cpi_value,-0.304725,1.000000


In [24]:
correlation_data = calculate_correlation(merged_data_with_cpi, 'NVDA')
correlation_data

,stock_price,cpi_value
stock_price,1.000000,-0.176158
cpi_value,-0.176158,1.000000
